In [2]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.utils import to_categorical
sys.path.append('../Metaheuristic Algorithms/')
from DESSA import DESSA
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Create an instance of the ImageDataGenerator for data normalization and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of data used for validation
    rotation_range=40,  # Data augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define the path to your dataset directory
train_directory = '/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Data/Dataset'

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [5]:
def create_vgg16_model(num_classes=4):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False  # Freeze the layers of VGG16

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
bounds = [(128, 128), (3, 5), (0.1, 0.5)]  # Example bounds, adjust these based on your earlier tests

dessa_optimizer = DESSA(lambda params: -create_vgg16_model().evaluate(validation_generator, steps=10)[1],
                        bounds, population_size=20, iterations=50)  # More iterations for better optimization

best_params, best_fitness, fitness_history = dessa_optimizer.optimize()
print("Best Parameters:", best_params)
print("Best Validation Accuracy:", -best_fitness)

10/10 [==============================] - 80s 8s/step - loss: 1.4618 - accuracy: 0.0156
Best Parameters: [128.           3.81201358   0.16051062]
Best Validation Accuracy: 0.512499988079071


In [11]:
model = create_vgg16_model(num_classes=4)
history = model.fit(
    train_generator,
    steps_per_epoch=10,  # Number of steps per epoch, adjust based on your data
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,  # Validation steps, adjust based on your validation data
    verbose=1
)

final_accuracy = history.history['val_accuracy'][-1]
print(f"Final model accuracy on validation set: {final_accuracy:.4f}")

Epoch 1/10
10/10 [==============================] - ETA: 0s - loss: 1.2237 - accuracy: 0.3938

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetic style of the plots
sns.set_style("whitegrid")


In [ ]:
# Extracting loss and accuracy from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Generating a list of epoch numbers to use in the x-axis of the plots
epochs = range(1, len(train_loss) + 1)

In [ ]:
plt.figure(figsize=(14, 5))

# Plotting training loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Training Loss', marker='o', color='blue')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o', color='red')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting training accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, label='Training Accuracy', marker='o', color='blue')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o', color='red')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()